In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os

In [76]:
# Define data transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        # transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [77]:
data_dir = 'data/Cars'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)
# Print some information about the datasets
print(f"Number of samples: {dataset_sizes}")
print(f"Classes: {class_names}")

['black', 'white']
Number of samples: {'train': 44, 'val': 11}
Classes: ['black', 'white']


In [78]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(class_names))
model.fc = nn.Linear(num_ftrs, 2)

In [79]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [80]:
import copy


def train_model(model, criterion, optimizer, num_epochs=15, patience=5):
    model = model.to(device)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    no_improve_epochs = 0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if best performance
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                no_improve_epochs = 0
            elif phase == 'val':
                no_improve_epochs += 1

        # Early stopping
        if no_improve_epochs >= patience:
            print(f'Early stopping triggered after epoch {epoch}')
            break

    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Usage
model = train_model(model, criterion, optimizer, num_epochs=30, patience=5)

Epoch 0/29
----------
train Loss: 0.7812 Acc: 0.6818
val Loss: 0.8651 Acc: 0.3636
Epoch 1/29
----------
train Loss: 0.9206 Acc: 0.5909
val Loss: 1.0517 Acc: 0.6364
Epoch 2/29
----------
train Loss: 1.0730 Acc: 0.6136
val Loss: 0.4695 Acc: 0.7273
Epoch 3/29
----------
train Loss: 0.5098 Acc: 0.7727
val Loss: 0.3933 Acc: 0.8182
Epoch 4/29
----------
train Loss: 0.5121 Acc: 0.8409
val Loss: 0.1755 Acc: 0.9091
Epoch 5/29
----------
train Loss: 0.4120 Acc: 0.8409
val Loss: 0.2359 Acc: 0.8182
Epoch 6/29
----------
train Loss: 0.6985 Acc: 0.8409
val Loss: 0.2274 Acc: 0.9091
Epoch 7/29
----------
train Loss: 0.7796 Acc: 0.7500
val Loss: 2.7604 Acc: 0.3636
Epoch 8/29
----------
train Loss: 0.2673 Acc: 0.8864
val Loss: 0.2617 Acc: 0.8182
Epoch 9/29
----------
train Loss: 0.7526 Acc: 0.7273
val Loss: 0.0910 Acc: 1.0000
Epoch 10/29
----------
train Loss: 0.3414 Acc: 0.8182
val Loss: 0.0643 Acc: 1.0000
Epoch 11/29
----------
train Loss: 0.5420 Acc: 0.8636
val Loss: 0.6411 Acc: 0.6364
Epoch 12/29
--

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = train_model(model, criterion, optimizer, num_epochs=15)

Epoch 0/14
----------
train Loss: 0.3235 Acc: 0.8955
val Loss: 0.0125 Acc: 1.0000
Epoch 1/14
----------
train Loss: 0.6313 Acc: 0.8358
val Loss: 0.0048 Acc: 1.0000
Epoch 2/14
----------
train Loss: 0.3315 Acc: 0.8955
val Loss: 0.1972 Acc: 0.8889
Epoch 3/14
----------
train Loss: 0.3097 Acc: 0.8657
val Loss: 0.2471 Acc: 0.8889
Epoch 4/14
----------
train Loss: 0.2137 Acc: 0.9403
val Loss: 0.0716 Acc: 1.0000
Epoch 5/14
----------
train Loss: 0.0924 Acc: 0.9701
val Loss: 0.0186 Acc: 1.0000
Epoch 6/14
----------
train Loss: 0.0536 Acc: 1.0000
val Loss: 0.0293 Acc: 1.0000
Epoch 7/14
----------
train Loss: 0.8900 Acc: 0.7612
val Loss: 0.1600 Acc: 0.8889
Epoch 8/14
----------
train Loss: 0.7596 Acc: 0.7910
val Loss: 0.0439 Acc: 1.0000
Epoch 9/14
----------
train Loss: 0.3135 Acc: 0.9254
val Loss: 0.0651 Acc: 1.0000
Epoch 10/14
----------
train Loss: 0.1176 Acc: 0.9403
val Loss: 0.0589 Acc: 1.0000
Epoch 11/14
----------
train Loss: 0.1321 Acc: 0.9552
val Loss: 0.0140 Acc: 1.0000
Epoch 12/14
--

In [81]:
torch.save(model.state_dict(), 'car_classifier03.pth')

In [82]:
#test the model with unseen images
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model.load_state_dict(torch.load('car_classifier03.pth', weights_only=False))
model.eval()
model = model.to(device)

In [83]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [84]:
from PIL import Image

def process_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image.to(device)

In [85]:
def predict(image_path):
    image = process_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    return class_names[predicted.item()]

In [97]:
image_path = 'data/Cars/test/0000353_02500_d_000019327.jpg'
prediction = predict(image_path)
print(f'The predicted class is: {prediction}')

The predicted class is: black
